# Data Engineering EDA Project

By Aommy, Austin, Ling, Tath (yippee)

-----------------------

## Installing Dependencies

### Minio Object Store dependencies

1. Install minio cli if not already installed.
2. Create client
3. Make bucket (if needed)

# no longer needed!

In [43]:
! pip install minio --quiet

In [44]:
# from minio import Minio

# # Create client with access and secret key.
# # client = Minio("s3.amazonaws.com", "ACCESS-KEY", "SECRET-KEY")

# # Create client with access key and secret key with specific region.
# client = Minio(
#     "localhost:9000",
#     access_key="ROOTNAME",
#     secret_key="CHANGEME123",
#     secure=False
# )

# if client.bucket_exists("eda"):
#     print("Bucket exists.")
# else:
#     client.make_bucket("eda")
#     print("Bucket created.")

#### DuckDB dependencies

In [45]:
!pip uninstall --quiet --yes malloy
!pip install --quiet --upgrade duckdb
!pip install --quiet jupysql==0.10.12
!pip install --quiet duckdb-engine

## Importing libraries

In [46]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import io
from urllib.request import urlopen
import zipfile
import os

In [47]:
!pip install --quiet yfinance --upgrade --no-cache-dir

---------------------

## Data Collection

### Data from Global Events

Installing data from http://data.gdeltproject.org/events/index.html for global event data!

In [48]:
# for y in range(19, 25):
#     for m in range(1, 13):
#         if m < 10:
#             m = "0" + str(m)
#         for d in range(1, 32):
#             if d < 10:
#                 d = "0" + str(d)
#             try:
#                 response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
#                 # Read data from response.
#             except:
#                 print("File doesn't exist!")
#                 try:
#                     data = urlopen(
#                         "http://data.gdeltproject.org/events/20{0}{1}{2}.export.CSV.zip".format(y,m,d),
#                     )
#                     print("Uploading file!")
#                     result = client.put_object(
#                         "eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), data, length=-1, part_size=10*1024*1024,
#                     )
#                     print(
#                         "created {0} object; etag: {1}, version-id: {2}".format(
#                             result.object_name, result.etag, result.version_id,
#                         ),
#                     )
#                 except:
#                     print("No file to download!")
#             finally:
#                 response.close()
#                 response.release_conn()

In [49]:
gdelt_headers = {
    "GlobalEventID": int,
    "Day": int,
    "MonthYear": int,
    "Year": int,
    "FractionDate": float,
    "Actor1Code": str,
    "Actor1Name": str,
    "Actor1CountryCode": str,
    "Actor1KnownGroupCode": str,
    "Actor1EthnicCode": str,
    "Actor1Religion1Code": str,
    "Actor1Religion2Code": str,
    "Actor1Type1Code": str,
    "Actor1Type2Code": str,
    "Actor1Type3Code": str,
    "Actor2Code": str,
    "Actor2Name": str,
    "Actor2CountryCode": str,
    "Actor2KnownGroupCode": str,
    "Actor2EthnicCode": str,
    "Actor2Religion1Code": str,
    "Actor2Religion2Code": str,
    "Actor2Type1Code": str,
    "Actor2Type2Code": str,
    "Actor2Type3Code": str,
    "IsRootEvent": bool,
    "EventCode": str,
    "EventBaseCode": str,
    "EventRootCode": str,
    "QuadClass": float,
    "GoldsteinScale": float,
    "NumMentions": float,
    "NumSources": float,
    "NumArticles": float,
    "AvgTone": float,
    "Actor1Geo_Type": float,
    "Actor1Geo_Fullname": str,
    "Actor1Geo_CountryCode": str,
    "Actor1Geo_ADM1Code": str,
    "Actor1Geo_Lat": float,
    "Actor1Geo_Long": object,
    "Actor1Geo_FeatureID": object,
    "Actor2Geo_Type": float,
    "Actor2Geo_Fullname": str,
    "Actor2Geo_CountryCode": str,
    "Actor2Geo_ADM1Code": str,
    "Actor2Geo_Lat": float,
    "Actor2Geo_Long": object,
    "Actor2Geo_FeatureID": object,
    "ActionGeo_Type": float,
    "ActionGeo_Fullname": str,
    "ActionGeo_CountryCode": str,
    "ActionGeo_ADM1Code": str,
    "ActionGeo_Lat": float,
    "ActionGeo_Long": object,
    "ActionGeo_FeatureID": object,
    "DateAdded": float,
    "SOURCEURL": str
}

In [50]:
len(gdelt_headers)

58

In [51]:
gdelt_headers.keys()

dict_keys(['GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_

### Data from YahooFinance

In [52]:
import yfinance as yf

companies = {
    "tech": ['AAPL', 'MSFT', 'NVDA'],
    "energy": ['XOM', 'CVX', 'BP', 'COP'],
    "food": ['KO', 'PEP', 'PG'],
    "health": ['JNJ', 'PFE', 'ABBV'],
    "finance": ['JPM', 'V', 'MA'],
    "agriculture": ['DE', 'CTVA', 'ADM'],
    "telecom": ['T', 'VZ', 'TMUS'],
    "materials": ['DOW', 'NEM', 'FCX']
}

In [53]:
# uploaded = False
# for sector in companies:
#     for stock in companies[sector]:
#         try:
#             response = client.get_object("eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock))
#             # Read data from response.
#         except:
#             print("File doesn't exist!")
#             data = yf.download(stock, start="2019-01-01", end="2024-10-15", progress=False)
#             result = client.put_object(
#                 "eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
#             )
#             print(
#                 "created {0} object; etag: {1}, version-id: {2}".format(
#                     result.object_name, result.etag, result.version_id,
#                 ),
#             )
#             uploaded = True
#         finally:
#             response.close()
#             response.release_conn()
# if (not uploaded):
#     print("No changes to make.")

### Data from Google Trend

In [54]:
! pip install pytrends --quiet

In [55]:
# import os
# import matplotlib.pyplot as plt
# from pytrends.request import TrendReq
# from minio import Minio
# from minio.error import S3Error

# companies = {
#     # Technology
#     'apple': ['Apple Inc.', 'Apple', 'AAPL', 'Apple Corporation'],
#     'microsoft': ['Microsoft', 'MSFT', 'Microsoft Corporation'],
#     'nvda': ['Nvidia', 'NVDA', 'Nvidia Corporation'],
#     # Energy/Oil
#     'exxon': ['Exxon Mobil', 'Exxon', 'Exxon Mobil Corporation'],
#     'chevron': ['Chevron', 'Chevron Corporation'],
#     'bp': ['BP', 'BP plc'],
#     'cop': ['ConocoPhillips', 'ConocoPhillips Company'],
#     # Consumer Goods/Food & Beverage
#     'coca_cola': ['Coca-Cola', 'Coca-Cola Company', 'KO'],
#     'pepsi': ['Pepsi', 'PepsiCo', 'PepsiCo Inc.'],
#     'pg': ['Procter & Gamble', 'Procter & Gamble Company', 'PG'],
#     # Healthcare
#     'jnj': ['Johnson & Johnson', 'Johnson & Johnson Company', 'JNJ'],
#     'pfe': ['Pfizer', 'Pfizer Inc.', 'PFE'],
#     'abbv': ['AbbVie', 'AbbVie Inc.', 'ABBV'],
#     # Financial Services
#     'jpm': ['JPMorgan Chase', 'JPMorgan Chase & Co.', 'JPM'],
#     'v': ['Visa', 'Visa Inc.', 'V'],
#     'ma': ['Mastercard', 'Mastercard Incorporated', 'MA'],
#     # Agriculture
#     'de': ['Deere & Company', 'Deere', 'DE'],
#     'ctva': ['Corteva', 'Corteva Inc.', 'CTVA'],
#     'adm': ['ADM', 'ADM Company', 'Archer Daniels Midland Company'],
#     # Telecommunications
#     't': ['AT&T', 'AT&T Inc.', 'T'],
#     'vz': ['Verizon', 'Verizon Communications', 'VZ'],
#     'tmus': ['T-Mobile', 'T-Mobile US', 'TMUS'],
#     # Materials
#     'dow': ['Dow Inc.', 'Dow', 'DOW'],
#     'nem': ['Newmont', 'Newmont Corporation', 'NEM'],
#     'fcx': ['Freeport-McMoRan', 'Freeport-McMoRan Inc.', 'FCX'],
# }

# platforms = {
#     'web': '',
#     'youtube': 'youtube',
#     'news': 'news',
#     'images': 'images',
# }

# # Initialize Pytrends
# pytrends = TrendReq(hl='en-US', tz=360)

# output_dir = 'data'
# os.makedirs(output_dir, exist_ok=True)
# image_dir = 'image'
# os.makedirs(image_dir, exist_ok=True)

# # Loop through each company and platform to get trends data
# for company, keywords in companies.items():
#     print(f"Fetching data for: {company}")
    
#     # Create a directory for the company
#     company_dir = os.path.join(output_dir, company)
#     os.makedirs(company_dir, exist_ok=True)

#     trends_data = {}  # Store trends for all platforms

#     # Fetch trends data for each platform
#     for platform, gprop in platforms.items():
#         pytrends.build_payload(kw_list=keywords, timeframe='2019-01-01 2024-10-15', geo='', gprop=gprop)
#         data = pytrends.interest_over_time()

#         # print(f"{platform} data:\n", data[keywords])

#         data.fillna(0, inplace=True)

#         # Sum all keyword trends for each platform
#         trends_data[platform] = data[keywords].sum(axis=1)

#         # Save platform data to CSV
#         csv_path = os.path.join(company_dir, f'{company}_{platform}.csv')
#         data.to_csv(csv_path)
#         print(f"Saved {csv_path}")

#     # Plot trends for each platform
#     plt.figure(figsize=(12, 8))
#     for platform, trend in trends_data.items():
#         plt.plot(trend.index, trend, label=platform)

#     plt.title(f'Search Trends for {company} (2019-2024)')
#     plt.xlabel('Date')
#     plt.ylabel('Search Interest')
#     plt.legend(loc='upper left')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.grid()

#     # Save the plot as an image
#     image_path = os.path.join(image_dir, f'{company}_search_trends.png')
#     plt.savefig(image_path)
#     plt.close()
#     print(f"Saved {image_path}")

In [56]:
# bucket_name = "eda"

# def upload_directory_to_minio(local_path, bucket, minio_path=""):
#     for root, _, files in os.walk(local_path):
#         for file in files:
#             local_file = os.path.join(root, file)
#             relative_path = os.path.relpath(local_file, local_path)
#             minio_file_path = os.path.join(minio_path, relative_path).replace("\\", "/")

#             try:
#                 client.fput_object(bucket, minio_file_path, local_file)
#                 print(f"Uploaded '{local_file}' to '{minio_file_path}'")
#             except S3Error as e:
#                 print(f"Failed to upload '{local_file}': {e}")

# # Upload 'data' and 'image' directories to MinIO under the 'google-trend' folder
# upload_directory_to_minio(output_dir, bucket_name, 'google-trend/data')
# upload_directory_to_minio(image_dir, bucket_name, 'google-trend/image')

# print("All files uploaded to MinIO under 'google-trend' folder.")


----------------------

## Data Processing

Convering .CSV.zip to .snappy.parquet!

In [57]:
# for y in range(19, 25):
#     for m in range(1, 13):
#         if m < 10:
#             m = "0" + str(m)
#         for d in range(1, 32):
#             if d < 10:
#                 d = "0" + str(d)
#             try:
#                 response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d))
#                 # Read data from response.
#             except:
#                 print("File doesn't exist! 20{0}/{1}/{2}".format(y,m,d))
#                 try:
#                     response = client.get_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
#                     client.fget_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), 
#                                        "./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
#                     with zipfile.ZipFile("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d),"r") as zip_ref:
#                         zip_ref.extractall("./data/gdelt/temp")
#                     data = pd.read_csv("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d), sep='\t', header=None, names=gdelt_headers.keys(), dtype=gdelt_headers, on_bad_lines='warn')
#                     result = client.put_object(
#                         "eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
#                     )
#                     print(
#                         "created {0} object; etag: {1}, version-id: {2}".format(
#                             result.object_name, result.etag, result.version_id,
#                         ),
#                     )
#                     os.remove("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d))
#                     os.remove("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
#                 except:
#                     print("This date does not exist!!!")

In [58]:
# data

----------------------

## Data Analysis

In [59]:
import duckdb as ddb

In [60]:
! pip install pyarrow --quiet
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [61]:
# %sql ROLLBACK

In [62]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# SPECIFY WHERE YOUR DDB IS !!!

`%sql duckdb:///<path to ur duckdb relative to where this .ipynb file is>`

In [63]:
%sql duckdb:///eda-ddb/eda-gdelt.ddb

No longer need to run the cell below! Already done with minIO part

In [64]:
# %%sql
# INSTALL httpfs;
# LOAD httpfs;
# SET s3_region='us-east-1';
# SET s3_url_style='path';
# SET s3_endpoint='localhost:9000';
# SET s3_access_key_id='ROOTNAME' ;
# SET s3_secret_access_key='CHANGEME123';
# SET s3_use_ssl = false;

In [65]:
# %%sql
# SELECT *
# FROM 's3://eda/gdelt-parquet/2019/01/01.snappy.parquet'

In [66]:
# %sql DROP TABLE gdelt

In [67]:
# %sql CREATE TABLE gdelt AS SELECT * FROM read_parquet('s3://eda/gdelt-parquet/2019/*/*.snappy.parquet', union_by_name=True);

In [68]:
# %sql SELECT * FROM gdelt

In [69]:
# %sql SELECT count(*) FROM gdelt

In [70]:
# %sql create table yfinance as select * from read_parquet("s3://eda/yfinance/*/*.snappy.parquet")

In [71]:
# %sql SELECT count(*) FROM yfinance

In [72]:
# %sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2020/*/*.snappy.parquet")

In [73]:
# %sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2021/*/*.snappy.parquet")

In [74]:
# %sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2022/*/*.snappy.parquet")

In [75]:
# %sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2023/*/*.snappy.parquet")

In [76]:
# %sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2024/*/*.snappy.parquet")

In [77]:
%sql SELECT count(*) from gdelt

,count_star()
0,260658282


In [78]:
%sql SELECT * FROM gdelt LIMIT 3;

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor2Code,...,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor2Geo_Type,Actor2Geo_CountryCode,ActionGeo_Type,ActionGeo_CountryCode,SOURCEURL
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,AGR,...,6.0,45.0,-1.070367,0.0,None,4.0,CA,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,USA,...,1.0,8.0,-1.333333,0.0,None,3.0,US,3.0,US,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,3.0,15.0,-1.112403,4.0,CA,0.0,None,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...


In [79]:
# %%sql
# ALTER TABLE gdelt DROP COLUMN DateAdded;
# ALTER TABLE gdelt DROP COLUMN ActionGeo_Lat;
# ALTER TABLE gdelt DROP COLUMN ActionGeo_Long;
# ALTER TABLE gdelt DROP COLUMN ActionGeo_FeatureID;
# ALTER TABLE gdelt DROP COLUMN ActionGeo_ADM1Code;
# ALTER TABLE gdelt DROP COLUMN ActionGeo_Fullname;
# ALTER TABLE gdelt DROP COLUMN Actor2Geo_ADM1Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Geo_Lat;
# ALTER TABLE gdelt DROP COLUMN Actor2Geo_Long;
# ALTER TABLE gdelt DROP COLUMN Actor2Geo_FeatureID;
# ALTER TABLE gdelt DROP COLUMN Actor2Geo_Fullname;
# ALTER TABLE gdelt DROP COLUMN Actor1Geo_ADM1Code;
# ALTER TABLE gdelt DROP COLUMN Actor1Geo_Lat;
# ALTER TABLE gdelt DROP COLUMN Actor1Geo_Long;
# ALTER TABLE gdelt DROP COLUMN Actor1Geo_FeatureID;
# ALTER TABLE gdelt DROP COLUMN Actor1Geo_Fullname;
# ALTER TABLE gdelt DROP COLUMN Actor1EthnicCode;
# ALTER TABLE gdelt DROP COLUMN Actor2EthnicCode;
# ALTER TABLE gdelt DROP COLUMN Actor1Religion1Code;
# ALTER TABLE gdelt DROP COLUMN Actor1Religion2Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Religion1Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Religion2Code;
# ALTER TABLE gdelt DROP COLUMN Actor1Type1Code;
# ALTER TABLE gdelt DROP COLUMN Actor1Type2Code;
# ALTER TABLE gdelt DROP COLUMN Actor1Type3Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Type1Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Type2Code;
# ALTER TABLE gdelt DROP COLUMN Actor2Type3Code;

In [80]:
%sql SELECT * FROM gdelt LIMIT 3;

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor2Code,...,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor2Geo_Type,Actor2Geo_CountryCode,ActionGeo_Type,ActionGeo_CountryCode,SOURCEURL
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,AGR,...,6.0,45.0,-1.070367,0.0,None,4.0,CA,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,USA,...,1.0,8.0,-1.333333,0.0,None,3.0,US,3.0,US,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,3.0,15.0,-1.112403,4.0,CA,0.0,None,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...


In [81]:
%%sql
select *
from INFORMATION_SCHEMA.COLUMNS
where TABLE_NAME='gdelt'

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,eda-gdelt,main,gdelt,GlobalEventID,1,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
1,eda-gdelt,main,gdelt,Day,2,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
2,eda-gdelt,main,gdelt,MonthYear,3,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
3,eda-gdelt,main,gdelt,Year,4,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
4,eda-gdelt,main,gdelt,FractionDate,5,None,YES,DOUBLE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
5,eda-gdelt,main,gdelt,Actor1Code,6,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
6,eda-gdelt,main,gdelt,Actor1Name,7,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
7,eda-gdelt,main,gdelt,Actor1CountryCode,8,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
8,eda-gdelt,main,gdelt,Actor1KnownGroupCode,9,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
9,eda-gdelt,main,gdelt,Actor2Code,10,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None


# Exploratory Data Analysis

In [1]:
! pip install plotly --quiet

import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
con = duckdb.connect('eda-ddb/eda-gdelt.ddb')

## Focusing on 2019 data

### Actor Insights

**Top Actors Involved in Events**
- We can find the top actors (both Actor1 and Actor2) involved in events and see the distribution of their involvement.

In [3]:
query = '''
SELECT Actor1Name, Actor2Name
FROM gdelt
WHERE Year = 2019;
'''

df_2019 = con.execute(query).fetchdf()

In [4]:
# Top Actor1 and Actor2 involved in the most events
actor1_count = df_2019['Actor1Name'].value_counts().reset_index().rename(columns={'index': 'Actor', 'Actor1Name': 'Actor'})
actor2_count = df_2019['Actor2Name'].value_counts().reset_index().rename(columns={'index': 'Actor', 'Actor2Name': 'Actor'})

In [5]:
top_actors = pd.concat([actor1_count, actor2_count]).groupby('Actor')['count'].sum().reset_index()
top_actors = top_actors.sort_values(by='count', ascending=False)

In [6]:
fig = px.bar(top_actors.head(10), x='Actor', y='count', 
             title='Top 10 Actors Involved in Events (2019)', 
             labels={'Count': 'Number of Events'})
fig.show()

### Geographical Insights

**Events by Country**
- Analyze the distribution of events based on countries for both Actor1 and Actor2, and the geographical location of the event (ActionGeo).

In [9]:
query = '''
SELECT Actor1Geo_CountryCode, Actor2Geo_CountryCode, ActionGeo_CountryCode
FROM gdelt
WHERE Year = 2019;
'''

df_2019 = con.execute(query).fetchdf()

In [12]:
actor1_geo_country = df_2019['Actor1Geo_CountryCode'].value_counts().reset_index().rename(columns={'index': 'Country', 'Actor1Geo_CountryCode': 'CountryCode'})
actor2_geo_country = df_2019['Actor2Geo_CountryCode'].value_counts().reset_index().rename(columns={'index': 'Country', 'Actor2Geo_CountryCode': 'CountryCode'})
action_geo_country = df_2019['ActionGeo_CountryCode'].value_counts().reset_index().rename(columns={'index': 'Country', 'ActionGeo_CountryCode': 'CountryCode'})

In [15]:
geo_distribution = pd.concat([actor1_geo_country, actor2_geo_country, action_geo_country]).groupby('CountryCode')['count'].sum().reset_index().sort_values(by='count', ascending=False)

fig = px.bar(geo_distribution.head(10), x='CountryCode', y='count', 
             title='Top 10 Countries by Event Distribution (2019)', 
             labels={'Count': 'Number of Events'})
fig.show()

### Event Characteristics

**Event Codes and QuadClass Analysis**
- Understanding the type of events based on `EventCode`, `EventBaseCode`, and `QuadClass`. These can reveal the nature of events (e.g., protests, military actions, etc.).

In [4]:
query = '''
SELECT EventCode, EventRootCode, QuadClass
FROM gdelt
WHERE Year = 2019;
'''
df_2019 = con.execute(query).fetchdf()

In [19]:
event_code_count = df_2019['EventCode'].value_counts().reset_index().rename(columns={'index': 'EventCode', 'EventCode': 'EventCode'})

In [23]:
# QuadClass distribution
quad_class_count = df_2019['QuadClass'].value_counts().reset_index().rename(columns={'index': 'QuadClass', 'QuadClass': 'QuadClass'})

In [22]:
# Plotting top 10 Event Codes
fig = px.bar(event_code_count.head(10), x='EventCode', y='count', 
             title='Top 10 Event Codes (2019)', 
             labels={'Count': 'Number of Events'})
fig.show()

# Plotting QuadClass distribution
fig = px.pie(quad_class_count, values='count', names='QuadClass', 
             title='Distribution of QuadClass (2019)')
fig.show()

**Goldstein Scale and Tone**
- Examine how event significance (GoldsteinScale) and tone (AvgTone) vary across different event types.

In [3]:
query = '''
SELECT GoldsteinScale, AvgTone, QuadClass
FROM gdelt
WHERE Year = 2019;
'''

df_2019 = con.execute(query).fetchdf()

In [13]:
aggregated_data = df_2019.groupby(['GoldsteinScale', 'QuadClass']).agg({'AvgTone': 'mean'}).reset_index()

In [14]:
fig = px.scatter(aggregated_data, x='GoldsteinScale', y='AvgTone', 
                 color='QuadClass', 
                 title='Goldstein Scale vs Average Tone (Aggregated, 2019)',
                 labels={'GoldsteinScale': 'Goldstein Scale', 'AvgTone': 'Average Tone', 'QuadClass': 'Event Type'})
fig.show()

### Temporal Analysis

**Events Over Time**
- Analyze how the number of events fluctuates over months or days of the year 2019.

In [20]:
query = '''
SELECT Year, MonthYear, Actor1Name, Actor2Name, EventCode
FROM gdelt
WHERE Year = 2019;
'''

df_2019 = con.execute(query).fetchdf()

In [21]:
df_2019['Month'] = df_2019['MonthYear'] % 100  # Extracting month from MonthYear
events_per_month = df_2019.groupby('Month')['EventCode'].count().reset_index()

fig = px.line(events_per_month, x='Month', y='EventCode', 
             title='Number of Events per Month in 2019', 
             labels={'EventCode': 'Number of Events'})
fig.show()